In [2]:
import asyncio
import nest_asyncio
import json
import hashlib
from datetime import datetime
from binascii import hexlify, unhexlify
from connectrum.client import StratumClient
from connectrum.svr_info import ServerInfo
from bitcash.cashaddress import Address

nest_asyncio.apply()

def extract_address(vout):
    spk = vout.get('scriptPubKey', {})
    if 'address' in spk:
        return spk['address']
    addresses = spk.get('addresses', [])
    if addresses:
        return addresses[0]
    return "Unknown/OP_RETURN"

def get_sh_manual(address_string):
    """
    Manually creates the ScriptHash. 
    This bypasses the 'obsolete protocol msgs' error by not letting 
    connectrum handle the address string.
    """
    try:
        addr = Address.from_string(address_string)
        # Create the ScriptPubkey (P2PKH or P2SH)
        # rr6tgh... is P2SH, so we handle the script construction
        script_hex = addr.scriptcode.hex()
        # SHA256 the script, then reverse (little-endian)
        sh = hashlib.sha256(unhexlify(script_hex)).digest()[::-1]
        return hexlify(sh).decode()
    except Exception as e:
        return None

async def get_addr_age(client, address):
    try:
        sh = get_sh_manual(address)
        if not sh: return 0, 100
        history = await client.RPC('blockchain.scripthash.get_history', sh)
        if not history: return 0, 100
        return len(history), (80 if len(history) < 5 else 20)
    except:
        return 0, 100

async def trace_bch_forensics(start_input, hops=5):
    # Public Fulcrum Node
    server = ServerInfo("BCH_Fulcrum", "bch.imaginary.cash", "s50002")
    client = StratumClient()
    trace_data = {"metadata": {"engine": "Gemini-Forensics-2026", "start": str(datetime.now())}, "steps": []}
    
    try:
        await client.connect(server)
        current_txid = None

        # Resolve Address to ScriptHash manually to avoid connectrum/pycoin bugs
        if start_input.startswith(('bitcoincash:', 'q', '1', 'p')):
            print(f"🔍 Manually resolving ScriptHash for: {start_input}")
            sh = get_sh_manual(start_input)
            if not sh:
                print("❌ Invalid Address format.")
                return
            
            history = await client.RPC('blockchain.scripthash.get_history', sh)
            if not history:
                print("❌ No history found.")
                return
            current_txid = history[-1]['tx_hash']
        else:
            current_txid = start_input

        print(f"✅ Connection Established. Mapping Path...\n")
        prev_time = None

        for i in range(hops):
            tx = await client.RPC('blockchain.transaction.get', current_txid, True)
            curr_time = tx.get('blocktime', tx.get('time', 0))
            time_str = datetime.fromtimestamp(curr_time).strftime('%Y-%m-%d %H:%M:%S')
            
            outputs = sorted(tx['vout'], key=lambda x: x['value'])
            is_final = len(outputs) < 2
            
            peel = outputs[0]
            change = outputs[-1]
            peel_addr = extract_address(peel)
            change_addr = extract_address(change)

            peel_tx_count, peel_risk = await get_addr_age(client, peel_addr)
            delta = curr_time - prev_time if prev_time else 0

            print(f"{'='*60}\n[STEP {i+1}] TRANSACTION DETAILS\n{'='*60}")
            print(f"🆔 TXID: {current_txid}")
            print(f"🎯 TARGET: {peel_addr} | VAL: {peel['value']} BCH")
            print(f"🛡️ RISK: {peel_risk}/100 | History: {peel_tx_count} txs")
            
            trace_data["steps"].append({
                "hop": i+1, "txid": current_txid, "timestamp": time_str,
                "sink": {"address": peel_addr, "value": peel['value']},
                "router": {"address": change_addr, "value": change['value']} if not is_final else None
            })

            if is_final: break
            
            prev_time = curr_time
            next_sh = get_sh_manual(change_addr)
            if not next_sh: break
            
            next_history = await client.RPC('blockchain.scripthash.get_history', next_sh)
            next_txs = [h['tx_hash'] for h in next_history if h['tx_hash'] != current_txid]
            
            if next_txs:
                current_txid = next_txs[0]
            else:
                break

        with open('bch_map.json', 'w') as f:
            json.dump(trace_data, f, indent=4)
        print(f"\n💾 Map saved to 'bch_map.json'")

    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        client.close()

# Run the trace
await trace_bch_forensics("bitcoincash:rr6tghm355fr8g30m0692nyd934dcrfjhs3ddl0fkc")

🔍 Manually resolving ScriptHash for: bitcoincash:rr6tghm355fr8g30m0692nyd934dcrfjhs3ddl0fkc
✅ Connection Established. Mapping Path...

[STEP 1] TRANSACTION DETAILS
🆔 TXID: 4bbdff0580c3166cbb62bb7f7b62a2dd5a2605601e7d8437955fe73c4fe59f7b
🎯 TARGET: Unknown/OP_RETURN | VAL: 0 BCH
🛡️ RISK: 100/100 | History: 0 txs
[STEP 2] TRANSACTION DETAILS
🆔 TXID: 0c814e5675a9d5c476a3a87d8acf542f481325d71a41da1d46f55b20018feb69
🎯 TARGET: bitcoincash:qzcal78gqjncjdnhxet0j780d9dsq7l3rgz8muxc25 | VAL: 1e-05 BCH
🛡️ RISK: 20/100 | History: 13 txs
[STEP 3] TRANSACTION DETAILS
🆔 TXID: 5c72080d42a6f79f4d77ebde7a351fd8987ccffeb1118a93f1e0e4945ca6e139
🎯 TARGET: bitcoincash:qq7sg4c986yj3de85ay4ntgfc4z0mf0scqwzknqmym | VAL: 0.001448 BCH
🛡️ RISK: 80/100 | History: 2 txs
[STEP 4] TRANSACTION DETAILS
🆔 TXID: 0c814e5675a9d5c476a3a87d8acf542f481325d71a41da1d46f55b20018feb69
🎯 TARGET: bitcoincash:qzcal78gqjncjdnhxet0j780d9dsq7l3rgz8muxc25 | VAL: 1e-05 BCH
🛡️ RISK: 20/100 | History: 13 txs
[STEP 5] TRANSACTION DETAILS
🆔 T